In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY= os.getenv("LANGSMITH_API_KEY")

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Llama3-8b-8192")
llm.invoke("Hi").content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [4]:
message = [{"role":"system", "content":"you are a helpful assistant"},
           {"role":"user", "content":"hi how are you?"}]

In [5]:
result = llm.invoke(message)
print(result.content)

Hi! I'm doing well, thank you for asking! I'm a helpful assistant, here to assist you with any questions or tasks you may have. How about you? How's your day going so far?


In [6]:
class Chatbot():
    def __init__(self, system = ""):
        self.system = system
        self.message = []
        if self.system:
            self.message.append({"role":"system", "content":system})
    
    def __call__(self, message):
        self.message.append({"role":"user","content":message})
        result = self.execute()
        self.message.append({"role":"assistant", "content":result})
        return result
    
    def execute(self):
        llm = ChatGroq(model_name="Llama3-8b-8192")
        result = llm.invoke(self.message)
        return result.content

In [7]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python to be sure to use floating point
syntax is necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi", "hello", "how are you", \n
you have to answer "hi", "hello", "i am good". 
""".strip()

In [8]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Vikas Chauhan\AppData\Local\Temp\ipykernel_14384\952804720.py:2: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)')


In [9]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [10]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
        blog_entry.created
        from
        blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
        blog_entry_fts match escape_fts(:q)
        order by
        blog_entry_fts.rank
        limit
        1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [11]:
def calculate(number):
    return eval(number)

In [12]:
calculate("2+2")

4

In [13]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [14]:
def query(question, max_turns = 5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [21]:
bot = Chatbot(prompt)

In [22]:
bot("Hi how are you?")

'Thought: The person is greeting me, I should respond with a polite greeting.\n\nAction: hi\n\nPAUSE'

In [23]:
bot("What is your job")

'Thought: My job is to answer questions and provide information based on my training data.\n\nAction: i am a question answering AI\n\nPAUSE'

In [24]:
query("hi")

Thought: The person is saying hi to me, I should respond in a friendly manner.

Action: hi

PAUSE


'Thought: The person is saying hi to me, I should respond in a friendly manner.\n\nAction: hi\n\nPAUSE'

In [25]:
query("Fifteen * twenty five")

Thought: I need to calculate the product of 15 and 25

Action: calculate: 15 * 25

PAUSE
 -- running calculate 15 * 25
Observation: 375
Answer: The result of the calculation is 375


'Answer: The result of the calculation is 375'

In [26]:
query("Has Simon written about AI?")

Thought: I should search Simon's blog for articles related to AI.

Action: simon_blog_search: AI

PAUSE
 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of 

'Thought: I seem to have been led on a bit of a tangent, but I think I\'ve found what I was looking for.\n\nAction: None (I think I\'ve got all I need)\n\nPAUSE\n\nAnswer: Simon has indeed written about AI, specifically about the term "Artificial Intelligence" and its implications.'